In [13]:
# それぞれの作者ごとに１つもファイルにまとめる
$ cat * > in.csv

SyntaxError: invalid syntax (<ipython-input-13-6a89dfb16213>, line 2)

In [30]:
# 名詞だけを取り出す
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

#docs = np.array([])
d = ""


with open("aiduyaiti/out.csv", "w") as f_out:
    with open("aiduyaiti/in.csv") as f_in:        
        line = f_in.readline()
        html_line = re.match("[^,]*", line)
        html = html_line.group(0)
        while line:
            search_line = re.match("([^,]*,){3}([^,]*),([^,]*)", line);
            if search_line is not None:
                if search_line.group(3) == "名詞":
                    d +=" "+search_line.group(2)
#                    print(search_line.group(2))
            line = f_in.readline()
            html_line = re.match("[^,]*", line)
            if html_line.group(0) != html:
                f_out.write("{}\n".format(d))
#                docs = np.insert(docs, d)
                d = ""
                html = html_line.group(0)
                

In [31]:
# 単語を特徴ベクトルに変換
count = CountVectorizer()
with open("aiduyaiti/out.csv") as f:
    lines = f.readlines()
    bag = count.fit_transform(lines)
    
print(count.vocabulary_)

{'人間': 130, 'やう': 72, 'つて': 45, 'こと': 24, '根気': 482, 'それ': 36, 'やつ': 73, '城壁': 273, '祭壇': 593, '神像': 591, '殿堂': 512, 'いつ': 9, 'この世': 25, 'ため': 40, '東洋': 477, '西洋': 673, '何処': 148, 'まま': 65, 'びた': 61, '腐肉': 635, '素質': 607, '遺族': 720, '友人': 229, 'もの': 70, 'せつ': 33, 'かく': 15, '粉々': 602, '地中': 270, '四角': 260, '御参り': 378, '故人': 426, '貞女': 692, '孝子': 313, '貧乏': 693, 'こんな': 27, '孝行': 314, 'むづかしい': 69, '建物': 366, '手わざ': 406, '時代': 455, 'ところ': 50, '一つ': 88, '風霜': 761, '曝露': 458, '平気': 359, '野火': 732, '火事': 539, '崩壊': 350, '立派': 600, '石碑': 585, '世間': 114, 'うち': 13, '土台': 265, '石垣': 583, '下積み': 110, '追慕': 708, '研究': 586, 'たち': 38, '搨拓': 421, '磨滅': 590, '村里': 474, '那全': 722, '山河': 348, '記念': 681, '無く': 540, 'いくら': 7, 'むかし': 68, '学識': 319, '手腕': 410, '情味': 391, '大官': 296, '政府': 425, '懐柔': 400, '平素': 360, '山水': 347, '眺め': 578, '好き': 309, '在任': 269, '同行': 245, '宇宙': 320, '開闢': 742, '初め': 206, 'ぶん': 63, '遊び': 714, 'わけ': 76, 'みんな': 67, '湮滅': 532, 'ここ': 22, '我々': 403, '魂魄': 767, 'あなた': 2, '功績': 213, '感化

In [22]:
print(bag.toarray())

[[0 0 1 ... 0 3 0]
 [0 0 0 ... 0 0 0]
 [2 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]


In [23]:
import pyprind
import pandas as pd
import os

In [24]:
# 単語の関連性を評価
np.set_printoptions(precision=2)
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

data_train = tfidf.fit_transform(count.fit_transform(lines)).toarray()
print(data_train)

[[0.   0.   0.04 ... 0.   0.12 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.06 0.03 0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.02 0.   0.02]]


In [25]:
# 学習

from sklearn.ensemble import RandomForestClassifier

label_train = [0, 1, 2, 3]

estimator = RandomForestClassifier()

estimator.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
# 元データを入れて試してみる

label_predict = estimator.predict(data_train)

print(label_predict)

[0 1 2 3]


In [27]:
print(estimator.score(data_train, label_train))

1.0
